In [1]:
import BoolODE as bo
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
from Perturbation import perturbation as pt
%autoreload 1 # not working

In [3]:
# user setting
model_definition = "HSC.txt" #"dyn-cycle.txt"

# perturbation parameter validation
# either of perturbation_transcription and perturbation_translation should be []
# either of perturbation_transcription and perturbation_translation should be float or list of dict
perturbed_transcription = 0.1 #[] #[{ 'g1': 10.0 }, { 'g2': 10.0 }]
perturbed_translation = [] #[{ 'Gata2': 0.5 }, { 'Cebpa': 0.3 }] # [{ 'g3': 5.0 }, { 'g4': 5.0 }] # 0.5

simulation_time = 1 # * 100 step
num_cells = 10 # = # of sampling_cells
sampling_time = [10, 30, 50] # 1~simulation_time*100-1?
output_dir = "Run1736" # change every simulation
output_filename = "PerturbationSampling.npy" # npy file
modeltype = "hill" # or heaviside

# default
do_parallel = False
sample_cells = False

In [4]:
# execution from module
E, gene_names, grn_mtx = pt.perturbation(
    model_definition, 
    simulation_time,
    num_cells,
    sampling_time,
    perturbed_transcription, 
    perturbed_translation,
    output_dir,
    output_filename,
    modeltype
)

 30%|███       | 3/10 [00:00<00:00, 25.64it/s]

Creating output folders
Starting simulations
Fixing rate parameters to defaults
{'n_Gata2': 10.0, 'n_Gata1': 10.0, 'n_Fog1': 10.0, 'n_Eklf': 10.0, 'n_Fli1': 10.0, 'n_Scl': 10.0, 'n_Cebpa': 10.0, 'n_Pu1': 10.0, 'n_cJun': 10.0, 'n_EgrNab': 10.0, 'n_Gfi1': 10.0, 'k_Gata2': 10.0, 'k_Gata1': 10.0, 'k_Fog1': 10.0, 'k_Eklf': 10.0, 'k_Fli1': 10.0, 'k_Scl': 10.0, 'k_Cebpa': 10.0, 'k_Pu1': 10.0, 'k_cJun': 10.0, 'k_EgrNab': 10.0, 'k_Gfi1': 10.0, 'sigmaH_Gata2': 10.0, 'sigmaH_Gata1': 10.0, 'sigmaH_Fog1': 10.0, 'sigmaH_Eklf': 10.0, 'sigmaH_Fli1': 10.0, 'sigmaH_Scl': 10.0, 'sigmaH_Cebpa': 10.0, 'sigmaH_Pu1': 10.0, 'sigmaH_cJun': 10.0, 'sigmaH_EgrNab': 10.0, 'sigmaH_Gfi1': 10.0, 'm_Gata2': 20.0, 'm_Gata1': 20.0, 'm_Fog1': 20.0, 'm_Eklf': 20.0, 'm_Fli1': 20.0, 'm_Scl': 20.0, 'm_Cebpa': 20.0, 'm_Pu1': 20.0, 'm_cJun': 20.0, 'm_EgrNab': 20.0, 'm_Gfi1': 20.0, 'l_x_Gata2': 10.0, 'l_x_Gata1': 10.0, 'l_x_Fog1': 10.0, 'l_x_Eklf': 10.0, 'l_x_Fli1': 10.0, 'l_x_Scl': 10.0, 'l_x_Cebpa': 10.0, 'l_x_Pu1': 10.0, 'l_

  0%|          | 0/10 [00:00<?, ?it/s]

Simulations took 0.407 s
Concating files took 0.06 s
Requested nClusters=1, not performing k-means clustering
Generating input files for pipline...
1. refNetwork
2. PseudoTime.csv
3. ExpressionData.csv
Input file generation took 0.04 s
BoolODE.py took 0.56s

 
 

Normal simulation ... Done
Starting perturbation control simulation...
Creating output folders
Starting simulations
Fixing rate parameters to defaults
{'n_Gata2': 10.0, 'n_Gata1': 10.0, 'n_Fog1': 10.0, 'n_Eklf': 10.0, 'n_Fli1': 10.0, 'n_Scl': 10.0, 'n_Cebpa': 10.0, 'n_Pu1': 10.0, 'n_cJun': 10.0, 'n_EgrNab': 10.0, 'n_Gfi1': 10.0, 'k_Gata2': 10.0, 'k_Gata1': 10.0, 'k_Fog1': 10.0, 'k_Eklf': 10.0, 'k_Fli1': 10.0, 'k_Scl': 10.0, 'k_Cebpa': 10.0, 'k_Pu1': 10.0, 'k_cJun': 10.0, 'k_EgrNab': 10.0, 'k_Gfi1': 10.0, 'sigmaH_Gata2': 10.0, 'sigmaH_Gata1': 10.0, 'sigmaH_Fog1': 10.0, 'sigmaH_Eklf': 10.0, 'sigmaH_Fli1': 10.0, 'sigmaH_Scl': 10.0, 'sigmaH_Cebpa': 10.0, 'sigmaH_Pu1': 10.0, 'sigmaH_cJun': 10.0, 'sigmaH_EgrNab': 10.0, 'sigmaH_Gfi1'

  0%|          | 0/10 [00:00<?, ?it/s]

...

try 2
Simulations took 0.598 s
Concating files took 0.06 s
Requested nClusters=1, not performing k-means clustering
Generating input files for pipline...
1. refNetwork
2. PseudoTime.csv
3. ExpressionData.csv
Perturbation output ...
Input file generation took 0.04 s
BoolODE.py took 0.73s


---------
EgrNab perturbation
---------
Creating output folders
Starting simulations
Fixing rate parameters to defaults
Setting trancsription perturbation parameters...
{'n_Gata2': 10.0, 'n_Gata1': 10.0, 'n_Fog1': 10.0, 'n_Eklf': 10.0, 'n_Fli1': 10.0, 'n_Scl': 10.0, 'n_Cebpa': 10.0, 'n_Pu1': 10.0, 'n_cJun': 10.0, 'n_EgrNab': 10.0, 'n_Gfi1': 10.0, 'k_Gata2': 10.0, 'k_Gata1': 10.0, 'k_Fog1': 10.0, 'k_Eklf': 10.0, 'k_Fli1': 10.0, 'k_Scl': 10.0, 'k_Cebpa': 10.0, 'k_Pu1': 10.0, 'k_cJun': 10.0, 'k_EgrNab': 10.0, 'k_Gfi1': 10.0, 'sigmaH_Gata2': 10.0, 'sigmaH_Gata1': 10.0, 'sigmaH_Fog1': 10.0, 'sigmaH_Eklf': 10.0, 'sigmaH_Fli1': 10.0, 'sigmaH_Scl': 10.0, 'sigmaH_Cebpa': 10.0, 'sigmaH_Pu1': 10.0, 'sigmaH_cJun'

  0%|          | 0/10 [00:00<?, ?it/s]

try 2
Simulations took 0.581 s
Concating files took 0.07 s
Requested nClusters=1, not performing k-means clustering
Generating input files for pipline...
1. refNetwork
2. PseudoTime.csv
3. ExpressionData.csv
Perturbation output ...
Input file generation took 0.04 s
BoolODE.py took 0.73s


---------
Gfi1 perturbation
---------
Creating output folders
Starting simulations
Fixing rate parameters to defaults
Setting trancsription perturbation parameters...
{'n_Gata2': 10.0, 'n_Gata1': 10.0, 'n_Fog1': 10.0, 'n_Eklf': 10.0, 'n_Fli1': 10.0, 'n_Scl': 10.0, 'n_Cebpa': 10.0, 'n_Pu1': 10.0, 'n_cJun': 10.0, 'n_EgrNab': 10.0, 'n_Gfi1': 10.0, 'k_Gata2': 10.0, 'k_Gata1': 10.0, 'k_Fog1': 10.0, 'k_Eklf': 10.0, 'k_Fli1': 10.0, 'k_Scl': 10.0, 'k_Cebpa': 10.0, 'k_Pu1': 10.0, 'k_cJun': 10.0, 'k_EgrNab': 10.0, 'k_Gfi1': 10.0, 'sigmaH_Gata2': 10.0, 'sigmaH_Gata1': 10.0, 'sigmaH_Fog1': 10.0, 'sigmaH_Eklf': 10.0, 'sigmaH_Fli1': 10.0, 'sigmaH_Scl': 10.0, 'sigmaH_Cebpa': 10.0, 'sigmaH_Pu1': 10.0, 'sigmaH_cJun': 

100%|██████████| 10/10 [00:00<00:00, 163.49it/s]

try 2
Simulations took 0.711 s
Concating files took 0.07 s
Requested nClusters=1, not performing k-means clustering
Generating input files for pipline...
1. refNetwork
2. PseudoTime.csv
3. ExpressionData.csv
Perturbation output ...
Input file generation took 0.05 s
BoolODE.py took 0.86s


In [8]:
gene_names

['Cebpa',
 'EgrNab',
 'Eklf',
 'Fli1',
 'Fog1',
 'Gata1',
 'Gata2',
 'Gfi1',
 'Pu1',
 'Scl',
 'cJun']

In [16]:
grn_mtx

array([[ 1.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  1.],
       [ 0.,  0.,  0., -1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  1.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1., -1.,  1.,  0., -1.,  0.,  0.],
       [ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0., -1., -1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.]])

In [51]:
pd.DataFrame(E[0,11,:,:])

,0,1,2,3,4,5,6,7,8,9,10
0,0.001228,0.025270,1.452094,0.292637,2.046821,1.951364,0.009977,0.001157,0.004692,1.751216,0.004982
1,0.006447,0.007104,0.878414,0.002502,2.124104,2.205830,0.003830,0.126561,0.003622,1.850792,0.012502
2,0.282448,0.013276,0.109167,1.742024,1.771670,1.538831,0.018015,0.706305,0.030857,1.785591,0.000551
3,0.001015,0.007488,1.576308,0.077863,1.783142,2.102115,0.010875,0.031208,0.052798,2.137598,0.003230
4,0.019559,0.004955,1.878801,0.485243,1.197779,1.350189,0.018329,0.007474,0.005279,1.867849,0.001027
5,0.043446,0.009105,1.473835,0.343274,1.177154,1.248541,0.042019,0.010826,0.052356,1.180597,0.043732
6,0.000479,0.033338,0.149951,1.806099,1.846503,2.401995,0.000009,0.004289,0.094300,1.775532,0.001470
7,0.003945,0.022089,1.653558,0.187541,2.588971,1.972416,0.032300,0.006212,0.005747,2.021844,0.006488
8,0.001691,0.018874,0.112563,2.082993,1.459610,1.501042,0.006357,0.010984,0.015880,1.530110,0.003072
9,0.019348,0.052454,0.198856,0.280384,0.336307,0.229472,0.012606,0.016133,0.003488,0.378859,0.043692


In [53]:
pd.DataFrame(E[0,1,:,:]) - pd.DataFrame(E[0,0,:,:])

,0,1,2,3,4,5,6,7,8,9,10
0,-2.350101e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.110223e-15,1.387779e-17,-2.020720e-09,-1.918103e-11,1.110223e-15,-3.191891e-16
1,-1.024689e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.782358e-12,-2.679974e-14,0.000000e+00,0.000000e+00
2,-1.147771e-01,3.201519e-11,2.785160e-11,1.558274e-10,1.565872e-10,2.135314e-07,4.850206e-10,-6.653631e-03,-2.349766e-04,2.140515e-07,2.379539e-07
3,-1.475180e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.346690e-15,-2.775558e-17,0.000000e+00,0.000000e+00
4,-1.336517e-04,-1.994932e-17,0.000000e+00,1.110223e-16,2.220446e-16,9.370282e-14,3.469447e-18,-3.019385e-08,-1.314422e-08,1.052491e-13,-7.403735e-14
5,-4.948228e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.938894e-18,-2.834900e-09,-6.189669e-10,4.440892e-16,-5.689893e-16
6,-2.502103e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.420348e-15,-6.245005e-16,0.000000e+00,0.000000e+00
7,-4.689775e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.540979e-18,0.000000e+00,0.000000e+00,0.000000e+00
8,-1.279612e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.328740e-09,-4.990147e-10,0.000000e+00,-1.734723e-18
9,-4.579628e-03,-2.101590e-12,6.133982e-15,5.884182e-15,5.995204e-15,6.317502e-12,0.000000e+00,-9.637842e-06,-4.810859e-06,6.629031e-12,-7.315927e-11


In [37]:
pd.DataFrame(E[0,2,:,:])

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
E.shape

(3, 12, 30, 11)